---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
len(states)

57

## Question 1

In [3]:
UT=pd.read_table('university_towns.txt', header = None)
UT.head(10)

,0
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]
5,Montevallo (University of Montevallo)[2]
6,Troy (Troy University)[2]
7,"Tuscaloosa (University of Alabama, Stillman Co..."
8,Tuskegee (Tuskegee University)[5]
9,Alaska[edit]


In [4]:
UT_State = UT.copy()
UT_State = UT_State[UT_State[0].str.contains(r"\[edit\]")]


Type  <class 'pandas.core.frame.DataFrame'>
Length  50


In [5]:
UT_State_List = UT_State[0].str.replace(r"\[.*\]","").str.strip().tolist()
len(UT_State_List)

50

In [7]:
UT[0]=UT[0].str.replace(r"\(.*\)","").str.replace(r"\[.*\]","").str.strip()
print("Type ",type(UT))
print("Length ",len(UT))
UT.head(10)

Type  <class 'pandas.core.frame.DataFrame'>
Length  567


,0
0,Alabama
1,Auburn
2,Florence
3,Jacksonville
4,Livingston
5,Montevallo
6,Troy
7,Tuscaloosa
8,Tuskegee
9,Alaska


In [8]:
UT.loc[0,0]

'Alabama'

In [9]:
UT.loc[0,0] in UT_State_List

True

In [10]:
data = []

for index, row in UT.iterrows():
    if(UT.loc[index,0] in UT_State_List):
        state = UT.loc[index,0]
        UT_State_List.remove(state)
    else:
        town = UT.loc[index,0]
        data.append([state,town])

In [11]:
unitowns = pd.DataFrame(data,columns=["State","RegionName"])
unitowns.shape

(517, 2)

In [12]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    UT=pd.read_table('university_towns.txt', header = None)
    
    UT_State = UT[UT[0].str.contains(r"\[edit\]")]
    UT_State_List = UT_State[0].str.replace(r"\[.*\]","").str.strip().tolist()
    
    UT[0]=UT[0].str.replace(r"\(.*","").str.replace(r"\[.*\]","").str.rstrip()
    
    data = []

    for index, row in UT.iterrows():
        if(UT.loc[index,0] in UT_State_List):
            state = UT.loc[index,0]
            UT_State_List.remove(state)
        else:
            town = UT.loc[index,0]
            data.append([state,town])

    unitowns = pd.DataFrame(data,columns=["State","RegionName"])
    
    return unitowns

In [15]:
df = get_list_of_university_towns()

In [16]:
cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')

Shape test:  Passed


In [17]:
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

Index test:  Passed


In [18]:
print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')

Column test:  Passed


In [19]:
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')

\n test:  Passed


In [20]:
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')

Trailing whitespace test: Passed


In [21]:
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')

"(" test: Passed


## Question 2

In [68]:
GDP = pd.read_excel('gdplev.xls',skiprows=220,header = None)
GDP.head(10)

,0,1,2,3,4,5,6,7
0,NaN,NaN,NaN,NaN,2000q1,10031.0,12359.1,NaN
1,NaN,NaN,NaN,NaN,2000q2,10278.3,12592.5,NaN
2,NaN,NaN,NaN,NaN,2000q3,10357.4,12607.7,NaN
3,NaN,NaN,NaN,NaN,2000q4,10472.3,12679.3,NaN
4,NaN,NaN,NaN,NaN,2001q1,10508.1,12643.3,NaN
5,NaN,NaN,NaN,NaN,2001q2,10638.4,12710.3,NaN
6,NaN,NaN,NaN,NaN,2001q3,10639.5,12670.1,NaN
7,NaN,NaN,NaN,NaN,2001q4,10701.3,12705.3,NaN
8,NaN,NaN,NaN,NaN,2002q1,10834.4,12822.3,NaN
9,NaN,NaN,NaN,NaN,2002q2,10934.8,12893.0,NaN


In [70]:
GDP=GDP[[4,5,6]]
print(GDP.shape)
GDP.head(10)

(66, 3)


,4,5,6
0,2000q1,10031.0,12359.1
1,2000q2,10278.3,12592.5
2,2000q3,10357.4,12607.7
3,2000q4,10472.3,12679.3
4,2001q1,10508.1,12643.3
5,2001q2,10638.4,12710.3
6,2001q3,10639.5,12670.1
7,2001q4,10701.3,12705.3
8,2002q1,10834.4,12822.3
9,2002q2,10934.8,12893.0


In [73]:
recession_start = []
for i in range(len(GDP) - 2):
    if(GDP.iloc[i][5] > GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] > GDP.iloc[i+2][5]):
        recession_start.append(GDP.iloc[i][4])

In [74]:
recession_start

['2008q3', '2008q4']

In [77]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls',skiprows=220,header = None)
    GDP=GDP[[4,5,6]]
    
    recession_start = []
    for i in range(len(GDP) - 2):
        if(GDP.iloc[i][5] > GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] > GDP.iloc[i+2][5]):
            recession_start.append(GDP.iloc[i][4])
    
    return recession_start[0]

In [92]:
recession_start = get_recession_start()
recession_start

'2008q3'

## Problem 3

In [89]:
recession_end = []
for i in range(len(GDP) - 2):
    if(GDP.iloc[i][5] < GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] < GDP.iloc[i+2][5]):
        recession_end.append(GDP.iloc[i+2][4])

In [93]:
recession_end = [i for i in recession_end if i > recession_start]

In [94]:
recession_end[0]

'2009q4'

In [95]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_end = []
    for i in range(len(GDP) - 2):
        if(GDP.iloc[i][5] < GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] < GDP.iloc[i+2][5]):
            recession_end.append(GDP.iloc[i+2][4])
    
    recession_end = [i for i in recession_end if i > recession_start]
    
    return recession_end[0]

In [97]:
recession_end = get_recession_end()
recession_end

'2009q4'

## Problem 4

### Approach 1

In [125]:
GDP[GDP[4] == recession_start].index[0]

34

In [126]:
GDP[GDP[4] == recession_end].index[0]

39

In [127]:
GDP.iloc[34:39]

,4,5,6
34,2008q3,14843.0,14891.6
35,2008q4,14549.9,14577.0
36,2009q1,14383.9,14375.0
37,2009q2,14340.4,14355.6
38,2009q3,14384.1,14402.5


In [128]:
recession = GDP.iloc[GDP[GDP[4] == recession_start].index[0]:GDP[GDP[4] == recession_end].index[0]]
recession[recession[5]==recession[5].min()].iloc[0][4]

'2009q2'

### Approach 2

In [112]:
recession = GDP[(GDP[4] > recession_start) & (GDP[4] < recession_end)]
recession

,4,5,6
35,2008q4,14549.9,14577.0
36,2009q1,14383.9,14375.0
37,2009q2,14340.4,14355.6
38,2009q3,14384.1,14402.5


In [119]:
recession[recession[5]==recession[5].min()].iloc[0][4]

'2009q2'

In [120]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession = GDP[(GDP[4] > recession_start) & (GDP[4] < recession_end)]
    
    return recession[recession[5]==recession[5].min()].iloc[0][4]

In [121]:
get_recession_bottom()

'2009q2'

## Problem 5

In [2]:
Zillow = pd.read_csv('City_Zhvi_AllHomes.csv')
Zillow.head(10)

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
5,18959,Las Vegas,NV,Las Vegas,Clark,6,121600.0,120900.0,120400.0,120300.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
6,54296,San Diego,CA,San Diego,San Diego,7,161100.0,160700.0,160400.0,160100.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
7,38128,Dallas,TX,Dallas-Fort Worth,Dallas,8,NaN,NaN,NaN,NaN,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
8,33839,San Jose,CA,San Jose,Santa Clara,9,224500.0,224900.0,225400.0,226100.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700
9,25290,Jacksonville,FL,Jacksonville,Duval,10,77500.0,77200.0,76800.0,76600.0,...,132000,132500,133100,133900,134900,136000,137200,138400,139500,140300


In [4]:
print(Zillow.columns)

Index(['RegionID', 'RegionName', 'State', 'Metro', 'CountyName', 'SizeRank',
       '1996-04', '1996-05', '1996-06', '1996-07',
       ...
       '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04',
       '2016-05', '2016-06', '2016-07', '2016-08'],
      dtype='object', length=251)


In [36]:
Zillow[['1996-04', '1996-05', '1996-06']].mean(axis=1)

0                  NaN
1        154666.666667
2        109466.666667
3         49833.333333
4         87700.000000
5        120966.666667
6        160733.333333
7                  NaN
8        224933.333333
9         77166.666667
10       263366.666667
11                 NaN
12                 NaN
13        83200.000000
14        60600.000000
15        95033.333333
16        67733.333333
17       123000.000000
18       163966.666667
19        53833.333333
20        99166.666667
21                 NaN
22        83900.000000
23        68100.000000
24        91533.333333
25       122100.000000
26        65333.333333
27        89633.333333
28       115666.666667
29        90333.333333
             ...      
10700              NaN
10701    151433.333333
10702              NaN
10703    103200.000000
10704     85600.000000
10705    128766.666667
10706              NaN
10707     55500.000000
10708     85000.000000
10709     85100.000000
10710    894166.666667
10711    108533.333333
10712      

In [8]:
Zillow[['1996-04', '1996-05', '1996-06']].mean(axis=0)

1996-04    120373.809229
1996-05    120454.175327
1996-06    120338.756388
dtype: float64

In [11]:
'1996-04'.split('-')

['1996', '04']

In [12]:
'04' >= '01'

True

In [18]:
List = '1996-01'.split('-')

In [19]:
Year = List[0]
Month = List[1]
print("Year ",Year)
print("Month ",Month)

Year  1996
Month  01


In [33]:
def month_to_quarter(String):
    List = String.split('-')
    Year = List[0]
    Month = List[1]
    Quarter=''
    if (Month >= '01') & (Month <= '03'):
        Quarter='q1'
    elif (Month >= '04') & (Month <= '06'):
        Quarter='q2'
    elif (Month >= '07') & (Month <= '09'):
        Quarter='q3'
    elif (Month >= '10') & (Month <= '12'):
        Quarter='q4'
    return Year+Quarter

In [34]:
print(month_to_quarter('1996-01'))
print(month_to_quarter('1996-02'))
print(month_to_quarter('1996-03'))
print(month_to_quarter('1996-04'))
print(month_to_quarter('1996-05'))
print(month_to_quarter('1996-06'))
print(month_to_quarter('1996-07'))
print(month_to_quarter('1996-08'))
print(month_to_quarter('1996-09'))
print(month_to_quarter('1996-10'))
print(month_to_quarter('1996-11'))
print(month_to_quarter('1996-12'))

1996q1
1996q1
1996q1
1996q2
1996q2
1996q2
1996q3
1996q3
1996q3
1996q4
1996q4
1996q4


In [56]:
Quarter_List = []
Months = ['1996-04','1996-05','1996-06','1996-07']
for month in Months:
    temp = month_to_quarter(month)
    if(temp not in Quarter_List):
        Quarter_List.append(temp)
Quarter_List

['1996q2', '1996q3']

In [58]:
Quarter_List = []
for month in Months:
    temp = month_to_quarter(month)
    print(Zillow.iloc[1][month])
    if(temp not in Quarter_List):
        Quarter_List.append(temp)
Quarter_List

155000.0
154600.0
154400.0
154200.0


['1996q2', '1996q3']

In [50]:
Zillow_column_names = Zillow.columns.tolist()
print(len(Zillow_column_names))
Zillow_column_names[0]

251


'RegionID'

In [49]:
Zillow_column_names[0][0].isdigit()

False

In [51]:
Zillow_column_month = [x for x in Zillow_column_names if x[0].isdigit()]
print(len(Zillow_column_month))
Zillow_column_month[0]

245


'1996-04'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"